In [2]:
from verl.utils import hf_tokenizer, hf_processor
import torch

In [3]:
model_name = "Qwen/Qwen2.5-VL-3B-Instruct"
processor = hf_processor(model_name)
tokenizer = hf_tokenizer(model_name)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [10]:
import verl.utils.torch_functional as verl_F
import torch
from verl.utils.model import compute_position_id_with_mask
response_with_chat_template='Abc def'
prompt_with_chat_template='ascas'                  
input_ids_response, attention_mask_response = verl_F.tokenize_and_postprocess_data(prompt=response_with_chat_template,
                                                                         tokenizer=tokenizer,
                                                                         max_length=10,
                                                                         pad_token_id=tokenizer.pad_token_id,
                                                                         left_pad=False,
                                                                         truncation='error')
input_ids_prompt, attention_mask_prompt = verl_F.tokenize_and_postprocess_data(prompt=prompt_with_chat_template,
                                                                         tokenizer=tokenizer,
                                                                         max_length=10,
                                                                         pad_token_id=tokenizer.pad_token_id,
                                                                         left_pad=True,
                                                                         truncation='error')
attention_mask_prompt=torch.zeros_like(input_ids_prompt) # All prompt will be masked



input_ids_prompt=input_ids_prompt[0]
attention_mask_prompt=attention_mask_prompt[0]
input_ids_response=input_ids_response[0]
attention_mask_response=attention_mask_response[0]
loss_mask_prompt = torch.zeros_like(attention_mask_prompt)


input_ids = torch.cat([input_ids_prompt, input_ids_response], dim=-1)
attention_mask = torch.cat([attention_mask_prompt, attention_mask_response], dim=-1)



position_ids_prompt = compute_position_id_with_mask(attention_mask_prompt)

response_length = input_ids_response.shape[0]
delta_position_id = torch.arange(1, response_length + 1, device=position_ids_prompt.device)



In [11]:
delta_position_id

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [15]:
position_ids_prompt[-1]=10

In [16]:
position_ids_response = position_ids_prompt[-1:] + delta_position_id

In [17]:
position_ids_response

tensor([11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

In [24]:
input_ids.shape

torch.Size([1, 1024])

In [ ]:
print(attention_mask[:10])  

In [19]:
new_input_ids, new_attention_mask, new_loss_mask=loss_mask(input_ids, attention_mask)
print(tokenizer.decode(new_input_ids[0][:30]))
print(new_attention_mask[0][:30])
print(new_loss_mask[0][:30])

DEBUG:input_ids.shape:torch.Size([1, 1024])
I love you<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>
tensor([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0])
tensor([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0])
